In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

data = load_iris()

clf = DecisionTreeClassifier(max_depth=3)
clf.fit(data.data, data.target)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [10]:
data.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [2]:
def rules(clf, features, labels, node_index=0):
    """Structure of rules in a fit decision tree classifier

    Parameters
    ----------
    clf : DecisionTreeClassifier
        A tree that has already been fit.

    features, labels : lists of str
        The names of the features and labels, respectively.

    """
    node = {}
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['name'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
    else:
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        node['name'] = '{} > {}'.format(feature, threshold)
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        node['children'] = [rules(clf, features, labels, right_index),
                            rules(clf, features, labels, left_index)]
    return node

In [3]:
rules(clf, data.feature_names, data.target_names)

{'name': 'petal width (cm) > 0.800000011920929',
 'children': [{'name': 'petal width (cm) > 1.75',
   'children': [{'name': 'petal length (cm) > 4.850000381469727',
     'children': [{'name': '0 of setosa, 0 of versicolor, 43 of virginica'},
      {'name': '0 of setosa, 1 of versicolor, 2 of virginica'}]},
    {'name': 'petal length (cm) > 4.949999809265137',
     'children': [{'name': '0 of setosa, 2 of versicolor, 4 of virginica'},
      {'name': '0 of setosa, 47 of versicolor, 1 of virginica'}]}]},
  {'name': '50 of setosa, 0 of versicolor, 0 of virginica'}]}

In [4]:
import json

r = rules(clf, data.feature_names, data.target_names)
with open('rules.json', 'w') as f:
    f.write(json.dumps(r))